# 不微调直接使用GLM

In [ ]:
import torch
from modelscope import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("ZhipuAI/glm-4-9b-chat", trust_remote_code=True)


# query = "你好"

# inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
#                                        add_generation_prompt=True,
#                                        tokenize=True,
#                                        return_tensors="pt",
#                                        return_dict=True
#                                        )

# inputs = inputs.to(device)
model = AutoModelForCausalLM.from_pretrained(
    "ZhipuAI/glm-4-9b-chat",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()
lora_model = model
# gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
# with torch.no_grad():
#     outputs = model.generate(**inputs, **gen_kwargs)
#     outputs = outputs[:, inputs['input_ids'].shape[1]:]
#     print(tokenizer.decode(outputs[0], skip_special_tokens=True))

2024-12-08 12:56:04.725645: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-08 12:56:04.765630: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-08 12:56:05.497620: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [2]:
def generate_response(inp, unique_id, max_new_tokens=2100):
    # inp = f"""你是一位杰出的医学专家，具备深厚的医学理论知识和丰富的临床实践经验，能够为用户提供权威、精准的医学咨询。请问：{inp}"""
    inp = f"""
    # 角色
    你是经验丰富的医学专家，擅长解答各类医学问题及医学选择题。
    # 工作任务
    针对用户的医学疑问，给出专业解答；
    对于医学选择题，参考如下输出示例，直接提供题目正确答案的选项即可。
    # 输出示例
    - 问:右侧桥小区囊性占位，囊肿可能大。偏肢体发育不良，巨脑回；右侧桥小脑区囊性占位，囊肿可能大.曾经治疗情况和效果：左手左脚没有右面灵活，爱往左歪脖，曾经感冒发烧39度抽过3、4次，抽时症状：身体僵硬呕吐。不知道是和囊肿有关还是和发育不良有关
      答:根据你描述的症状和病史，右侧桥小区囊性占位可能是导致偏肢体发育不良和巨脑回的原因之一。囊肿的大小可能也会对症状产生影响。\n\n至于感冒发烧抽搐的症状，这可能与囊肿有关，也可能与发育不良有关。囊肿对中枢神经系统的正常功能可能会有影响，导致肌肉僵硬和呕吐等症状。\n\n然而，请注意，我只是凭借你提供的信息做出初步推断，而不能提供确切的诊断。为了明确病因和确定最佳治疗方案，我建议你尽快咨询专业的医生进行详细的检查和进一步的评估。他们可以根据你的具体情况给出准确的诊断和建议。
    - 用户问题:以下哪个选项是治疗高血压的首选药物？
      A. 地巴唑 B. 硝苯地平 C. 洛汀新 D. 氢氯噻嗪
      输出:答案是：B
    # 注意事项
    确保解答准确无误，避免提供未经证实的医学信息。
    # 问题
    {inp}
    """
    # 与训练使用的template保持一致
    try:
        prompt = inp
        inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                               add_generation_prompt=True,
                                               tokenize=True,
                                               return_tensors="pt",
                                               return_dict=True
                                               )
        inputs = inputs.to(device)
        params = {"max_length": max_new_tokens, "do_sample": True, "top_k": 1}
        with torch.no_grad():
            outputs = lora_model.generate(**inputs, **params)
            outputs = outputs[:, inputs['input_ids'].shape[1]:]
            # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
            response = tokenizer.decode(outputs[0], skip_special_tokens=True, )

        
        # prompt = f"User: {inp}\nAssistant: "
        # inputs = tokenizer(prompt, return_tensors="pt").to(lora_model.device)
        # params = {
        #     "max_new_tokens": max_new_tokens,
        #     "top_p": 0.95,
        #     "temperature": 0.8,
        #     "do_sample": True
        # }
        # with torch.no_grad():
        #     outputs = lora_model.generate(**inputs, **params)
        # prompt_leng = len(inputs[0])
        # output = outputs[0][prompt_leng:]
        # response = tokenizer.decode(output, skip_special_tokens=True)
        run_info = {
            "unique_id": unique_id,
            "model_inp": prompt,
            "gen_parm": params

        }
        return response, run_info
    except:
        return None, {}

import hashlib
import json

def generate_unique_code(input_string):
    sha256_hash = hashlib.sha256()
    sha256_hash.update(input_string.encode('utf-8'))
    unique_code = sha256_hash.hexdigest()
    return unique_code

log_file = "./run_log.jsonl" # 推理日志
ans_file = "./model_logits_glm_prompt_2000.jsonl" # 推理结果

# prompt_lst = [["简单阐述下当前汽车的发展趋势","subjective"],["为什么中国大力推行新能源汽车","subjective"]]
prompt_lst = []
with open("eval_only_query.jsonl", "r", encoding='utf-8') as fw:
    for line in fw.readlines():
        line = eval(line)
        prompt_list = []
        prompt_list.append(line["query"])
        prompt_list.append(line["query_type"])
        prompt_lst.append(prompt_list)

for prompt, query_type in prompt_lst:
    unique_id = generate_unique_code(prompt)
    # print保留unique_id进行数据追溯
    response, run_info = generate_response(prompt, unique_id)
    if response is None:
        continue
    run_info["answer"] = response
    # 保存运行日志
    with open(log_file, "a", encoding='utf-8') as fw:
        fw.write(json.dumps(run_info, ensure_ascii=False)+"\n")
        # fw.write(json.dumps(run_info, ensure_ascii='utf-8')+"\n")

    # 保存模型输结果
    answer_info = {
        "query": prompt,
        "query_type": query_type,
        "answer": response
        }
    with open(ans_file, "a", encoding='utf-8') as fw:
        fw.write(json.dumps(answer_info, ensure_ascii=False)+"\n")

    print(f"answer_info: {answer_info}")


answer_info: {'query': 'Question: A normal heart rate for an adult at rest is:\nOptions: (A) 60-80 bpm.. (B) 60-100 bpm.. (C) 60-90 bpm.. (D) 60-110 bpm..\n', 'query_type': 'objective', 'answer': '\n答案是：A'}
answer_info: {'query': '问题：下列哪项原因不引起萎缩____\n选项：(A) 肾盂积水。(B) 慢性肝淤血。(C) 垂体功能低下。(D) 四氯化碳中毒。\n', 'query_type': 'objective', 'answer': '\n答案是：C'}
answer_info: {'query': '身体僵直,腿无力手不抖,是帕金森吗', 'query_type': 'subjective', 'answer': '\n答：身体僵直和腿无力手不抖可能是帕金森病的症状之一，但仅凭这些症状不能确诊。帕金森病是一种慢性神经系统疾病，其特征包括震颤、肌肉僵硬、运动迟缓和姿势平衡障碍。\n\n为了确定是否患有帕金森病，需要通过详细的病史询问、神经系统检查以及可能的影像学检查和血液检查。如果怀疑自己可能患有帕金森病，建议尽快咨询专业的医生进行诊断和治疗。'}
answer_info: {'query': '我去做妇科检查说子宫下垂我想知道什么原因才会造成子宫下垂呢？', 'query_type': 'subjective', 'answer': '\n子宫下垂，医学上称为子宫脱垂，是指子宫从正常位置沿阴道下降，甚至脱出阴道口外。造成子宫下垂的原因可能包括：\n\n1. 分娩损伤：特别是经阴道分娩，尤其是难产或产程过长，可能导致盆底肌肉和筋膜的损伤。\n2. 盆底肌肉松弛：随着年龄的增长，女性体内的雌激素水平下降，可能导致盆底肌肉松弛，从而增加子宫下垂的风险。\n3. 腹部压力增加：长期咳嗽、便秘、举重等导致腹部压力增加的活动，也可能导致子宫下垂。\n4. 肥胖：体重过重可能增加腹部压力，导致子宫下垂。\n5. 慢性疾病：如慢性咳嗽、尿失禁等慢性疾病也可能导致子宫下垂。\n6. 手术：某些腹部或盆腔手术，如阑尾炎手术，也可能影响盆底肌肉和筋膜，导

In [ ]:
inp = 'Question: A normal heart rate for an adult at rest is:\nOptions: (A) 60-80 bpm.. (B) 60-100 bpm.. (C) 60-90 bpm.. (D) 60-110 bpm..\n'
label = 'objective'
unique_id = 1
generate_response(inp, unique_id, max_new_tokens=2000)

## 测试

In [10]:
def generate_response(inp, unique_id, max_new_tokens=300):
    inp = f"""你是一位杰出的医学专家，具备深厚的医学理论知识和丰富的临床实践经验，能够为用户提供权威、精准的医学咨询。请问：{inp}"""
    # 与训练使用的template保持一致
    try:
        prompt = inp
        inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                               add_generation_prompt=True,
                                               tokenize=True,
                                               return_tensors="pt",
                                               return_dict=True
                                               )
        inputs = inputs.to(device)
        params = {"max_length": max_new_tokens, "do_sample": True, "top_k": 1}
        with torch.no_grad():
            outputs = model.generate(**inputs, **params)
            outputs = outputs[:, inputs['input_ids'].shape[1]:]
            # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(response)

        
        # prompt = f"User: {inp}\nAssistant: "
        # inputs = tokenizer(prompt, return_tensors="pt").to(lora_model.device)
        # params = {
        #     "max_new_tokens": max_new_tokens,
        #     "top_p": 0.95,
        #     "temperature": 0.8,
        #     "do_sample": True
        # }
        # with torch.no_grad():
        #     outputs = lora_model.generate(**inputs, **params)
        # prompt_leng = len(inputs[0])
        # output = outputs[0][prompt_leng:]
        # response = tokenizer.decode(output, skip_special_tokens=True)
        run_info = {
            "unique_id": unique_id,
            "model_inp": prompt,
            "gen_parm": params

        }
        return response, run_info
    except:
        return None, {}



inp = "Question: Sauna use, sometimes referred to as \"sauna bathing,\" is characterized by short-term passive exposure to extreme heat. This exposure elicits mild hyperthermia – an increase in the body's core temperature – that induces a thermoregulatory response involving neuroendocrine, cardiovascular, and cytoprotective mechanisms that work together to restore homeostasis and condition the body for future heat stressors… In recent decades, sauna bathing has emerged as a means to increase lifespan and improve overall health, based on compelling data from observational, interventional, and mechanistic studies. Of particular interest are the findings from studies of participants in the Kuopio Ischemic Heart Disease Risk Factor (KIHD) Study, an ongoing prospective population-based cohort study of health outcomes in more than 2,300 middle-aged men from eastern Finland, which identified strong links between sauna use and reduced death and disease… The KIHD findings showed that men who used the sauna two to three times per week were 27 percent less likely to die from cardiovascular-related causes than men who didn't use the sauna.[2] Furthermore, the benefits they experienced were found to be dose-dependent: Men who used the sauna roughly twice as often, about four to seven times per week, experienced roughly twice the benefits – and were 50 percent less likely to die from cardiovascular-related causes.[2] In addition, frequent sauna users were found to be 40 percent less likely to die from all causes of premature death. These findings held true even when considering age, activity levels, and lifestyle factors that might have influenced the men's health.[2]... The KIHD also revealed that frequent sauna use reduced the risk of developing dementia and Alzheimer's disease in a dose-dependent manner. Men who used the sauna two to three times per week had a 66 percent lower risk of developing dementia and a 65 percent lower risk of developing Alzheimer's disease, compared to men who used the sauna only one time per week… The health benefits associated with sauna use extended to other aspects of mental health, as well. Men participating in the KIHD study who used the sauna four to seven times per week were 77 percent less likely to develop psychotic disorders, regardless of the men's dietary habits, socioeconomic status, physical activity, and inflammatory status (as measured by C-reactive protein)…Exposure to high temperature stresses the body, eliciting a rapid, robust response. The skin and core body temperatures increase markedly, and sweating ensues. The skin heats first, rising to 40°C (104°F), and then changes in core body temperature occur, rising slowly from 37°C (98.6°F, or normal) to 38°C (100.4°F) and then rapidly increasing to 39°C (102.2°F)…  Cardiac output, a measure of the amount of work the heart performs in response to the body's need for oxygen, increases by 60 to 70 percent, while the heart rate (the number of beats per minute) increases and the stroke volume (the amount of blood pumped) remains unchanged.[5] During this time, approximately 50 to 70 percent of the body's blood flow is redistributed from the core to the skin to facilitate sweating. The average person loses approximately 0.5 kg of sweat while sauna bathing.[11] Acute heat exposure also induces a transient increase in overall plasma volume to mitigate the decrease in core blood volume. This increase in plasma volume not only provides a reserve source of fluid for sweating, but it also acts like the water in a car's radiator, cooling the body to prevent rapid increases in core body temperature and promoting heat tolerance… Repeated sauna use acclimates the body to heat and optimizes the body's response to future exposures, likely due to a biological phenomenon known as hormesis, a compensatory defense response following exposure to a mild stressor that is disproportionate to the magnitude of the stressor. Hormesis triggers a vast array of protective mechanisms that not only repair cell damage but also provide protection from subsequent exposures to more devastating stressors… The physiological responses to sauna use are remarkably similar to those experienced during moderate- to vigorous-intensity exercise. In fact, sauna use has been proposed as an alternative to exercise for people who are unable to engage in physical activity due to chronic disease or physical limitations.[13]\n\nWhich of the following is a likely paragraph that follows this excerpt in the article?\nOptions: (A) A paragraph on a protein that facilitates intracellular function in response to heat.. (B) A paragraph on increased heart attacks in Eskimo populations.. (C) A recap of Finland’s water polo team excellence.. (D) A study on rats exposed to high levels of heat..\n"
unique_id = 1
generate_response(inp, unique_id, max_new_tokens=1000)
print()


(A) A paragraph on a protein



## 二次处理结果

In [1]:
import torch
from modelscope import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

# tokenizer = AutoTokenizer.from_pretrained("ZhipuAI/glm-4-9b-chat", load_in_8bit=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("ZhipuAI/glm-4-9b-chat", trust_remote_code=True)


# query = "你好"

# inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
#                                        add_generation_prompt=True,
#                                        tokenize=True,
#                                        return_tensors="pt",
#                                        return_dict=True
#                                        )

# inputs = inputs.to(device)
model = AutoModelForCausalLM.from_pretrained(
    "ZhipuAI/glm-4-9b-chat",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()
lora_model = model
# gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
# with torch.no_grad():
#     outputs = model.generate(**inputs, **gen_kwargs)
#     outputs = outputs[:, inputs['input_ids'].shape[1]:]
#     print(tokenizer.decode(outputs[0], skip_special_tokens=True))

2024-12-08 10:52:52.393627: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-08 10:52:52.498818: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-08 10:52:53.469052: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [2]:
def fix_response(query, answer, label, max_new_token=2000):
    if label == "objective":
        try:
            inp = f"""
            # 角色
            你是一个优秀的医生，可以根据问题从题解中的总结出正确的选择题选项

            # 问题
            {query}

            # 选择题解析文本
            {answer}

            # 回答格式
            答案是：
            """

            prompt = inp
            inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                                   add_generation_prompt=True,
                                                   tokenize=True,
                                                   return_tensors="pt",
                                                   return_dict=True
                                                   )
            inputs = inputs.to(device)
            params = {"max_length": max_new_token, "do_sample": True, "top_k": 1}
            with torch.no_grad():
                outputs = lora_model.generate(**inputs, **params)
                outputs = outputs[:, inputs['input_ids'].shape[1]:]
                # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # prompt = f"User: {inp}\nAssistant: "
            # inputs = tokenizer(prompt, return_tensors="pt").to(lora_model.device)
            # params = {
            #     "max_new_tokens": 10,
            #     "top_p": 0.95,
            #     "temperature": 0.8,
            #     "do_sample": True
            # }
            # with torch.no_grad():
            #     outputs = lora_model.generate(**inputs, **params)
            # prompt_leng = len(inputs[0])
            # output = outputs[0][prompt_leng:]
            # response = tokenizer.decode(output, skip_special_tokens=True)
            return response
        except:
            return answer
    else:
#         try:
#             inp = f"""
#             # 角色
#             你是一个优秀的医生，可以根据输入的问题和回答，对回答的内容进行优化。

#             # 问题
#             {query}

#             # 回答
#             {answer}

#             # 功能
#             1. 去除回答中与问题不相关内容。
#             2. 整理回答中的语言，使其语义完整连贯。
#             3. 仅输出整理后的回答文本。
#             """
#             # inp = f"""将文本：{answer}进行优化，使其语言通顺完整，去除无意义的套话"""
#             prompt = f"User: {inp}\nAssistant: "
#             inputs = tokenizer(prompt, return_tensors="pt").to(lora_model.device)
#             params = {
#                 "max_new_tokens": 200,
#                 "top_p": 0.95,
#                 "temperature": 0.8,
#                 "do_sample": True
#             }
#             with torch.no_grad():
#                 outputs = lora_model.generate(**inputs, **params)
#             prompt_leng = len(inputs[0])
#             output = outputs[0][prompt_leng:]
#             response = tokenizer.decode(output, skip_special_tokens=True)
#             return response
#         except:
#             return answer
        return answer

In [ ]:
query = '问题：下列哪项原因不引起萎缩____\n选项：(A) 肾盂积水。(B) 慢性肝淤血。(C) 垂体功能低下。(D) 四氯化碳中毒。\n'
answer = '选项（D）四氯化碳中毒不会直接导致器官或组织的萎缩。四氯化碳中毒主要影响肝脏，会导致肝细胞坏死和肝纤维化，进而引起肝硬化，而不是萎缩。其他选项中提到的原因都是可以导致相应器官或组织发生萎缩的因素：\n\n- (A) 肾盂积水会导致肾脏缺血，最终可能导致肾实质萎缩。\n- (B) 慢性肝淤血可导致肝细胞脂肪变性和坏死，从而引起肝萎缩。\n- (C) 垂体功能低下会导致腺垂体分泌激素减少，如生长激素、促性腺激素等，这会影响相关靶器官的功能，长期下去可能会导致这些器官的萎缩。 \n\n因此，正确答案是（D）。四氯化碳中毒不是引起萎缩的原因。但请注意，长期的肝损伤（如由四氯化碳引起的肝损伤），在疾病进展过程中，可能伴随有肝实质的萎缩'
label = 'objective'
max_new_token = 2000
fix_response(query, answer, label, max_new_token)

## 生成二次结果

In [3]:
import json

with open("model_logits_glm_2000.jsonl", "r", encoding='utf-8') as f:
    for line in f.readlines():
        line = eval(line)
        query = line["query"]
        label = line["query_type"]
        answer = line["answer"]

        result = fix_response(query, answer, label)
        print(result)

        # 保存模型输结果
        answer_info = {
            "query": query,
            "query_type": label,
            "answer": result
            }
        with open("model_logits_fix_glm_2000.jsonl", "a", encoding='utf-8') as fw:
            fw.write(json.dumps(answer_info, ensure_ascii=False)+"\n")


答案是：(B) 60-100 bpm.

答案是：没有一个选项是不引起萎缩的原因。根据题解，所有选项（A、B、C、D）都可能导致器官或组织的萎缩。

身体僵直、腿无力、手不抖这些症状可能与多种疾病有关，帕金森病是其中之一，但并不一定是帕金森病。帕金森病是一种慢性神经系统退行性疾病，主要症状包括震颤、肌肉僵直、运动迟缓和姿势平衡障碍。

以下是一些可能导致类似症状的疾病或情况：

1. 帕金森病：这是最常见的导致上述症状的疾病。
2. 脑血管疾病：如中风或脑梗塞，可能导致肢体无力或运动障碍。
3. 脊髓疾病：如脊髓损伤或脊髓炎，可能导致肢体无力。
4. 药物副作用：某些药物，如抗抑郁药、抗精神病药等，可能导致肌肉僵硬和震颤。
5. 中毒：如重金属中毒（如铅、汞）也可能导致类似的症状。
6. 神经肌肉疾病：如肌萎缩侧索硬化症（ALS）等。

为了确定具体原因，需要通过以下方式进行诊断：

- 详细询问病史和症状。
- 进行体格检查，包括神经系统检查。
- 进行影像学检查，如MRI或CT扫描，以排除其他可能的疾病。
- 有时可能需要进行血液检查或脑电图（EEG）等。

建议您尽快就医，由专业的医生进行全面的评估和诊断。不要自行诊断或治疗，以免延误病情。

子宫下垂，医学上称为子宫脱垂，是指子宫从正常位置沿阴道下降，甚至脱出阴道口外的一种现象。造成子宫下垂的原因可能包括以下几点：

1. **分娩损伤**：尤其是经阴道分娩，特别是难产或产程过长，可能导致盆底肌肉和筋膜的损伤，从而引起子宫下垂。

2. **年龄因素**：随着年龄的增长，女性体内的雌激素水平下降，盆底肌肉和筋膜会逐渐松弛，增加子宫下垂的风险。

3. **肥胖**：肥胖会增加腹部压力，对盆底肌肉造成额外的负担，可能导致子宫下垂。

4. **长期慢性咳嗽**：慢性咳嗽会增加腹压，对盆底肌肉造成压力，可能导致子宫下垂。

5. **长期站立或重体力劳动**：这些活动会增加腹压，对盆底肌肉造成压力。

6. **盆腔肿瘤**：如子宫肌瘤、卵巢囊肿等，可能会压迫盆腔器官，导致子宫下垂。

7. **手术史**：如子宫切除术、阑尾切除术等，可能会损伤盆腔结构，增加子宫下垂的风险。

8. **遗传因素**：家族中如果有子宫下垂的病史，可能增加个人患病的风险。

了解子宫下垂的原因后，以下是一些可能的检查和治疗方法：